In [1]:
import re
import ujson
import dask.bag as db
from dask.diagnostics import ProgressBar
from bs4 import BeautifulSoup

pbar = ProgressBar()
pbar.register()


def extract(blob):
    return blob['dataList']
        

def remove_empty(data_list):
    return data_list


def valid_columns(data):
    return {'detalleNorma': data['detalleNorma'], 'idTramite': data['idTramite']}


def extract_text(data):
    try:
        return {
            'idTramite': data['idTramite'],
            'parsedText': BeautifulSoup(data['detalleNorma'], "html.parser").getText().strip()
        }
    except:
        return None

def remove_nones(data):
    return data is not None


filename = "example.dat"
bag = db.read_text(filename, blocksize=10000000)

df = bag.map(ujson.loads).map(extract).filter(remove_empty).map(valid_columns).map(extract_text).filter(remove_nones)

In [2]:
from pyfreeling import Analyzer

def tokenize(data):
    try:
        analyzer = Analyzer(config='/usr/local/Cellar/freeling/4.0/share/freeling/config/es-ar.cfg')
        tokens = []
        xml_root = analyzer.run(data['parsedText'].encode('utf-8'), 'nec')
        for element in xml_root.iter():
            if element.tag == 'token':
                tokens.append(dict(element.attrib))
    except:
        print(data['parsedText'])

    return {'idTramite': data['idTramite'], 'tokens': tokens}
tokens = df.map(tokenize)

In [3]:
tokens.map(ujson.dumps).to_textfiles('{}.*.dat'.format(filename))

[########################################] | 100% Completed |  4min 50.5s
